In [ ]:
"""
Created on Wed Aug  4 17:50:06 2021

@author: amol
"""

# set TF GPU memory growth so that it doesn't hog everything at once
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from main import *
debug_mode = True  # faster experiments for debugging

In [ ]:
#Prepare the DataFrame that will be used downstream
dp = DataPreparer()
dp.createPCAs()
dp.sparsePCAs()
dp.zmixOrthogonalPCAs()
df = dp.getDataframe()

df.to_csv('PCA_data.csv', index=False)

# currently passing dp eventually we want to abstract all the constants into 1 class
dm = DataManager(df, dp)

In [ ]:
print(df)

In [ ]:
'''
1. Run the GP Experiments
'''
import time

start=time.time()
exprExec = run_gp_experiments(dm, debug_mode=debug_mode)
print(f'duration: {time.time()-start}')

In [ ]:
'''
2. Run the Simple DNN Experiments/
'''
exprExec = run_simple_dnn_experiments(dm, debug_mode=debug_mode)

In [ ]:
'''
3. Run the PCDNN_v1 Experiments
'''
exprExec = run_pcdnn_v1_experiments(dm, debug_mode=debug_mode)

In [ ]:
'''
4. Run the PCDNN_v2 Experiments
'''
exprExec = run_pcdnn_v2_experiments(dm, debug_mode=debug_mode)

In [ ]:
# My version:
import model_analyzer.model_analysis as model_analysis
import pdb

import importlib
importlib.reload(model_analysis)

bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
print(f'\nexperimentSettings: {experimentSettings}')
print(f'\nbestModel.input_shape: {bestModel.input_shape}')
inspector = model_analysis.ModelInspector(exprExec.modelFactory, dm)

In [ ]:
inspector.plot_permutation_feature_importance()

In [ ]:
inspector.plot_partial_dependence()

### Print & Record Linear Embeddings Output

In [ ]:
import numpy as np

def inspect_PCA(X):
    PCA_dict = {}
    PCA_dict['X'] = X
    PCA_dict['Var'] = PCA_dict['X'].var(axis=0)
    
    # sort by explained variance
    sort_idx = np.argsort(PCA_dict['Var'])[::-1]
    PCA_dict['X'] = PCA_dict['X'][:, sort_idx]
    PCA_dict['Var'] = PCA_dict['Var'][sort_idx]
    PCA_dict['CumVar'] = np.add.accumulate(PCA_dict['Var'])
    return PCA_dict


X, Y,rom,zmix = dm.getAllData()
FullDataset = inspect_PCA(X)
try:    
    linearAutoEncoder = exprExec.modelFactory.getLinearEncoder()
    X = linearAutoEncoder.predict(FullDataset['X'])
    DNN_PCA = inspect_PCA(X)
except KeyError:
    print('No Linear Auto Encoder!')

In [ ]:
#dm.createDataset
import copy
dm_PurePCA = copy.deepcopy(dm)
dm_PurePCA.createTrainTestData(dataSetMethod='PurePCA_randomequalflamesplit',
                               numCpvComponents=2, ipscaler=None, opscaler=None)

X,Y,rom,zmix = dm_PurePCA.getAllData()
PurePCA = inspect_PCA(X)
print(PurePCA['Var'])
print(DNN_PCA['Var'])

plt.plot(PurePCA['CumVar']/FullDataset['Var'].sum(), color='r')
plt.plot(DNN_PCA['CumVar']/FullDataset['Var'].sum(), color='b')
plt.title('PCA Total Variance Comparison')

In [ ]:
import pandas as pd

df = pd.DataFrame()
df = df.append({'x' : 1, 'y' : 2, 'z': 3}, ignore_index=True)

In [ ]:
print(df)
df = df.append({'x' : 1, 'y' : 2, 'z': 3}, ignore_index=True)